#### Important note:
* in order for this code to run fully, you will need folders called `CSVs/` and `Plots/` in the local directory.

# Setup
## Functions
Here we define some functions.

### Miscellaneous functions:
* CDF (for use in generating matrices)
* epsilon needed (for use later)

In [16]:
import numpy as np
rng1 = np.random.default_rng()


def eps_needed(p_succ,eps_list):
    '''
    Input:
    - p_succ (float) between 0 and 1: probability of success
    - eps_list (list) is the list (of epsilons) we want to take a quantile from

    Output:
    - eps_needed (float)
    '''
    return np.percentile(eps_list, 100 * p_succ)

    
def CDF(pvec):
    '''
    Input: 
    - pvec (list) = probabilities vector

    Output:
    - CDFvec (list) = vector of cumulative probabilities

    '''
    CDFvec=[]
    s=sum(pvec)
    if s != 1: # ensure normalisation
        for p in pvec:
            p /= s
    for i in pvec:
        if len(CDFvec)==0:
            CDFvec.append(i)
        else:
            CDFvec.append(CDFvec[-1]+i)
    return CDFvec

### For generating matrices
* normal
* uniform
* countsketch
* discrete (old)
* discrete
* constant
* Q of QR
* sparse (beta) - modify the Generation function to use this to generate vectors

In [17]:
def rand_normal_matrix(m: int,n: int,normalvariables: list):
    '''
    Input:
    - m (int) = # rows
    - n (int) = # cols
    - normalvariables (list) = [mean, SD]

    Output:
    - matrix (numpy.ndarray) = random normal matrix of shape (m,n)

    Example:
    >>> rand_normal_matrix(2,5,[0,1])
    '''    
    no_entries = m*n
    M=rng1.normal(size=[int(m),int(n)],loc = normalvariables[0], scale=normalvariables[1])
    #M *= 1/np.sqrt(m)
    return M

def rand_uniform_matrix(no_rows:int=1,no_cols:int=1,low=0,high=1):
    '''
    Input:
    - no_rows (int) = # rows
    - no_cols (int) = # cols
    - low (float) = minimum possible value
    - high (float) = maximum possible value
    
    Output:
    - Matrix (numpy.ndarray) where each entry is Uniform(low,high) of shape (no_rows,no_cols)

    Example usage:
    >>> rand_uniform_matrix(no_rows=2,no_cols=2, 0, 1)
    '''
    return rng1.uniform(low,high,(no_rows,no_cols))

def rand_countsketch_matrix(no_rows: int,no_cols: int):
    '''
    Input:
    - num_rows (int) = # rows
    - num_cols (int) = # rows

    Output:
    - numpy.ndarray = CountSketch matrix of shape (num_rows, num_cols).
    
    Example usage:
    >>> rand_countsketch_matrix(no_rows,no_cols)
    '''
    # Decide which rows to pick for each column
    rowpicks = np.floor(rng1.uniform(low=0,high=no_rows,size=(1,no_cols))).astype(int)
    # Decide which of {-1,1} to pick for all nz entries at once
    pm1picks = np.round(rng1.uniform(low=0,high=1,size=(1,no_cols)),0)
    pm1picks = pm1picks*2-1
    # Apply
    matrix = np.zeros((no_rows,no_cols))
    matrix[rowpicks,range(no_cols)]=pm1picks
    return matrix

def old_rand_discrete_matrix(xvec,pvec=None,size=(1,1)):
    '''
    Recommended example:
    >>> rand_discrete_matrix(xvec=[1,2,5],
                             CDFvec=CDF([0.1,0.6,0.3]),
                             size=(3,2)
                             )
    '''
    if len(xvec) != len(pvec):
        raise TypeError("Value and probability vectors must be of equal sizes!")
    CDFvec=CDF(pvec)
    UMat = rng1.uniform(0,1,(size[0],size[1]))
    for i, high in enumerate(CDFvec):
        low = CDFvec[i - 1] if i - 1 >= 0 else 0
        UMat[(low < UMat) & (UMat < high)] = xvec[i]
    output = UMat
    return output

def rand_discrete_matrix(no_rows,no_cols,xvec, pvec=None):
    '''
    Input:
    - no_rows (int) = # rows
    - no_cols (int) = # cols
    - xvec (list) = possible values for entries
    - pvec (list) = probabilities of each of the values in xvec, respectively

    Output:
    - matrix (numpy.ndarray) = random discrete matrix of shape (m,n)

    Assumes a uniform distribution if pvec is not supplied.

    Example usage:
    >>> rand_discrete_matrix(3,5,[1,-1],[0.3,0.7])
    '''
    if pvec is None:
        pvec = [1/len(xvec) for _ in range(len(xvec))]
    if len(xvec) != len(pvec):
        raise RuntimeError("Value and probability vectors must be of equal sizes!")
    if np.sum(pvec) != 1:
        raise RuntimeError("Probabilities must add to 1!")
    matrix = rng1.choice(a=xvec,size=(no_rows,no_cols),p=pvec)
    return matrix

def constant_matrix(nrows:int=1,ncols:int=1,constant:float=1):
    '''
    Input:
    - nrows
    - ncols
    - constant
    Output:
    - np.ndarray nrows x ncols, all entries = constant
    '''
    M=rand_uniform_matrix(nrows,ncols,constant,constant)
    return M

def rand_QR_matrix(no_rows: int, no_cols: int, fliptype: int = 1):
    '''
    Input:
    - no_rows (int) = # rows
    - no_cols (int) = # cols
    - fliptype (int)
    Fliptype description based on Gram-Schmidt:
    1) all signs + (sign flip is removed)
    2) biased +- (based on numpy QR)
    3) random +- (based on removing sign flip, then adding random flips)

    Output:
    - matrix (numpy.ndarray) = random Q matrix of shape (no_rows,no_cols)

    Example usage:
    >>> rand_QR_matrix(no_rows=2,no_cols=3,fliptype=1)
    '''
    # Generate Gaussian matrix to then QR factorise
    NormalMat = rand_normal_matrix(no_rows,no_cols,[0,1])

    if no_rows < no_cols: # Flip 1 of 2
        NormalMat = NormalMat.T
    
    # Factorise as QR
    Q,R=np.linalg.qr(NormalMat,mode='reduced')

    # (Possibly) change sign flips
    if fliptype == 1:
        newQ = Q @ np.diag(np.sign(np.diag(R)))
    elif fliptype == 2:
        newQ = Q
    elif fliptype == 3:
        noflipQ = Q @ np.diag(np.sign(np.diag(R))) # remove the biased flip first
        # Generate some random sign flips and apply
        pm1vec = rand_discrete_matrix(1,no_rows,[-1,1],pvec=[0.5,0.5])
        pm1array=pm1vec[0]
        newQ = noflipQ @ np.diag(pm1array)
    else:
        raise NameError('Unknown flip type - should be 1, 2, or 3')
    
    if no_rows < no_cols: #Flip 2 of 2
        newQ = newQ.T
    
    return np.sqrt(no_cols/no_rows) * newQ

def rand_sparse_normal_matrix(no_rows: int,no_cols: int,normalvariables: list,prob_zero_entry: float):
    '''
    Input:
    - no_rows (int): Number of rows in the matrix.
    - no_cols (int): Number of columns in the matrix.
    - normalvariables (list): [mean, standard deviation] of normal entries
    - prob_zero_entry (float): the probability that a given entry is 0, instead of being from N(mean, std)

    Output:
    - array: sparsely sampled array
    
    Example usage:
    >>> rand_sparse_normal_matrix(no_rows=10,no_cols=2,[0,1],0.9)
    '''
    matrix = np.array(rand_normal_matrix(no_rows,no_cols,normalvariables))
    discrete0s = np.array(rand_discrete_matrix(no_rows,no_cols,xvec=[0,1],pvec=[prob_zero_entry,1-prob_zero_entry]))
    return matrix * discrete0s

## Generation
### Parameters
We now create the parameters for the generation events

In [18]:
# Parameters for how many data to generate
no_x_vecs = 200 # number of vectors to generate per matrix
no_S_mats = 200 # number of matrices to generate per k

# Parameters for m = input dimension, k = output dimension
m = 2000 # size of data
logkmin=np.log10(m)-1/3
logkmax=np.log10(m)
logknum=5
KLIST=np.logspace(logkmin,logkmax,logknum,base=10)

# Matrix choices to loop over soon
choicevec=['normal','uniform','countsketch','QR1','QR2','QR3','discretePM1']#,'constant1']

Analysis parameters:

Since the analysis begins within the generation at the checking of 

In [19]:
delta=0.05 # failure probability

### Main Generation Code
### Steps
1. Loop over logarithmically spaced values of $k$ (as generated before)

5. Loop over different distributions $\Pi_2$: choices include `['normal','uniform','countsketch','QR1','QR2','QR3','constant0']`, stored in `choicevec`.

1. Fix $m$ (the ambient dimension of the data) as a large constant. In this project, `2000` is used.

1. Create some random vectors $\vec{x}$ where $x_i \sim \Pi(\pi_{1,1},\pi_{1,2}) =$ i.i.d. (or equivalently, $\vec{x} \sim N(0,I_n)$).

    a. We will use $\Pi$ as a Normal distribution, since data is likely to follow a Normal distribution by the CLT
    
    b. Normalise these vectors by re-defining $\vec{x} = \frac{\vec{x}}{|| \vec{x} || }$

    NB. These will be stored in an array for ease of data storage and multiplication.

2. Create some random matrices $S$ with $S_{ij} \sim \Pi_2(\pi_{2,1}, \pi_{2,2})$ 

    a. Set any relevant parameters to $\Pi_2$, for example mean and variance for a normal distribution.

    b. Generate the matrices with a previously built function and any relevant parameters
    
3. Save the distribution of $$\varepsilon_\text{implied} := \left| \frac{\| S \vec{x}\|−\|\vec{x}\|}{\|\vec{x}\|} \right| = \left|  \|S\vec{x}\|−1 \right|$$ *for every pair $(S,\vec{x})$*.

    * There should be `no_x_vecs` $\times$ `no_S_mats` values of $\varepsilon_{\text{implied},k,{\Pi_2}}$ here

    * This is saved into an array, then a CSV.

4. Save the distribution of '`eps_needed`', the $1-\delta$ quantile of `eps_needed`, for some pre-determined failure probability $\delta$ - i.e. the $\varepsilon$ for which $1-\delta$ of the samply distributed $\varepsilon$ are below that.

    * This is saved into a CSV which is read in the plotting code.

NB. When generating matrices, first the code checks to see if the step (3.) and (4.) outputs have already been saved for exactly the same parameters. If not, a message is displayed and they are re-generated. If they did, a different message is displayed, and nothing new is generated for that matrix and parameter set.

#### Reference list:
* Vectors are `(m,1)` and there are `no_x_vecs` of them
* Matrices are `(k,m)` (squish from $m$ dim to $k$) and there are `no_S_mats` of them

In [20]:
for choice in choicevec:
    # Generate a matrix with the rows as random vectors of length m
    xvec_mat=rand_normal_matrix(no_x_vecs,m,[0,1])
    # Normalise
    xvec_mat/=np.linalg.norm(xvec_mat,axis=1)[:,np.newaxis]

    # Loop
    EPSNEEDEDFORK=[]
    for k in KLIST:
        k=int(np.round(k,0)) # need k to be an integer!
        Smats = None
        Dont_Generate_Eps=False

        #paths
        pathEpslist=f'CSVs/EpsList_kMAX{logkmin}by{logkmax}by{logknum}m{m}for{no_x_vecs}xvecs{no_S_mats}SmatswithDelta{delta}for{choice}.csv'
        pathEpsForK=f'CSVs/EpsForK_k{logkmin}by{logkmax}by{logknum}m{m}for{no_x_vecs}xvecs{no_S_mats}SmatswithDelta{delta}for{choice}.csv'

        #Generation
    # Known
        # Check which type it is, and then generate accordingly if CSV data does not already exist.
        if choice == 'normal':
            try:
                _=np.loadtxt(pathEpslist, delimiter=',')
                _=np.loadtxt(pathEpsForK, delimiter=',')
                Dont_Generate_Eps=True
                print(f"{choice} matrices already exist")
            except:
                print(f"{choice} matrices didn't exist yet")
                Smats = np.array([rand_normal_matrix(k,m, [0,1 / np.sqrt(k)]) for _ in range(no_S_mats)])
        elif choice == 'countsketch':
            try:
                _=np.loadtxt(pathEpslist, delimiter=',')
                _=np.loadtxt(pathEpsForK, delimiter=',')
                Dont_Generate_Eps=True
                print(f"{choice} matrices already exist")
            except:
                print(f"{choice} matrices didn't exist yet")
                Smats = np.array([rand_countsketch_matrix(k,m) for _ in range(no_S_mats)])
        elif choice == 'uniform':
            try:
                _=np.loadtxt(pathEpslist, delimiter=',')
                _=np.loadtxt(pathEpsForK, delimiter=',')
                Dont_Generate_Eps=True
                print(f"{choice} matrices already exist")
            except:
                print(f"{choice} matrices didn't exist yet")
                Smats = np.array([rand_uniform_matrix(k,m,0,1) for _ in range(no_S_mats)])
        elif choice.startswith('constant'):
            try:
                _=np.loadtxt(pathEpslist, delimiter=',')
                _=np.loadtxt(pathEpsForK, delimiter=',')
                Dont_Generate_Eps=True
                print(f"{choice} matrices already exist")
            except:
                print(f"{choice} matrices didn't exist yet")
                Smats = np.array([constant_matrix(k,m,int(choice[-1])) for _ in range(no_S_mats)])
        elif choice.startswith('QR'):
            try:
                _=np.loadtxt(pathEpslist, delimiter=',')
                _=np.loadtxt(pathEpsForK, delimiter=',')
                Dont_Generate_Eps=True
                print(f"{choice} matrices already exist")
            except:
                print(f"{choice} matrices didn't exist yet")
                Smats = np.array([rand_QR_matrix(k,m,fliptype=int(choice[-1])) for _ in range(no_S_mats)])
        elif choice == 'discretePM1':
            try:
                _=np.loadtxt(pathEpslist, delimiter=',')
                _=np.loadtxt(pathEpsForK, delimiter=',')
                Dont_Generate_Eps=True
                print(f"{choice} matrices already exist")
            except:
                print(f"{choice} matrices didn't exist yet")
                Smats = np.array([rand_discrete_matrix(k,m,xvec=[-1,1]) for _ in range(no_S_mats)])
    # Unknown
        else:
            raise RuntimeError('Unknown choice')
        if not Dont_Generate_Eps:
            # For each choice, generate the corresponding EpsNeededforK list for the K list.
            mat_applied = Smats@(np.transpose(xvec_mat))
            eps_array = np.abs(np.linalg.norm(mat_applied,axis=1)-1) #since 1 = norm(xvec)
            eps_list = eps_array.flatten()

            e_needed = eps_needed(1-delta,eps_list) # operate on the failure probability and the refreshed eps_list = epslist(k,matrix type) to calculate e_needed for this k value.
            EPSNEEDEDFORK.append(e_needed)
    #-------
        
    choice=choice.capitalize()
    print(f"{choice} matrices generated.")

    if not Dont_Generate_Eps:
        np.savetxt(pathEpslist, eps_list, delimiter=',') # This is in fact only saving the eps_list for MAX k - i.e. when k=m.
        np.savetxt(pathEpsForK, EPSNEEDEDFORK, delimiter=',')
        print(f'{choice} saved.')
        print('')

print('Generation complete.')

normal matrices already exist
normal matrices already exist
normal matrices already exist
normal matrices already exist
normal matrices already exist
Normal matrices generated.
uniform matrices already exist
uniform matrices already exist
uniform matrices already exist
uniform matrices already exist
uniform matrices already exist
Uniform matrices generated.
countsketch matrices already exist
countsketch matrices already exist
countsketch matrices already exist
countsketch matrices already exist
countsketch matrices already exist
Countsketch matrices generated.
QR1 matrices already exist
QR1 matrices already exist
QR1 matrices already exist
QR1 matrices already exist
QR1 matrices already exist
Qr1 matrices generated.
QR2 matrices already exist
QR2 matrices already exist
QR2 matrices already exist
QR2 matrices already exist
QR2 matrices already exist
Qr2 matrices generated.
QR3 matrices already exist
QR3 matrices already exist
QR3 matrices already exist
QR3 matrices already exist
QR3 mat